In [ ]:
# === SETUP & IMPORTS ===
import carla
import time
import threading
import pandas as pd
import random
import os
import sys
import numpy as np
import cv2
import torch
import torch.nn as nn

# === MODEL DEFINITION ===
class SensorClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.net(x)

# === MODEL LOAD ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SensorClassifier().to(DEVICE)
model.load_state_dict(torch.load('aml_gps_classifier.pth', map_location=DEVICE))
model.eval()

# === CARLA SETUP ===
os.makedirs('video_Aml', exist_ok=True)
sys.path.append('D:/carla/PythonAPI')
sys.path.append('D:/carla/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
map = world.get_map()
blueprints = world.get_blueprint_library()
spawn_points = map.get_spawn_points()

for a in world.get_actors().filter('*vehicle*'): a.destroy()
for s in world.get_actors().filter('*sensor*'): s.destroy()

start_index = int(input("\nEnter index for START location: "))
spoof_index = int(input("Enter index for SPOOFED GPS location: "))
start_transform = spawn_points[start_index]
spoof_location = spawn_points[spoof_index].location

vehicle_bp = blueprints.filter('vehicle.lincoln.mkz_2020')[0]
vehicle = world.try_spawn_actor(vehicle_bp, start_transform)
spectator = world.get_spectator()
spectator.set_transform(carla.Transform(
    vehicle.get_transform().transform(carla.Location(x=-5, z=2)),
    vehicle.get_transform().rotation))

vehicle.set_autopilot(True)

spoofing_active = False
spoofed_latitude = spoof_location.x
spoofed_longitude = spoof_location.y
frame_id = 0
video_fps = 20
video_writers = {}
data_records = []
sensor_data = { 'gnss': None, 'imu': None }
sensor_list = []

def follow_vehicle():
    while vehicle.is_alive:
        transform = vehicle.get_transform()
        spectator.set_transform(carla.Transform(
            transform.transform(carla.Location(x=-6, z=3)),
            transform.rotation))
        time.sleep(0.05)

threading.Thread(target=follow_vehicle, daemon=True).start()

# === SENSOR CALLBACKS ===
def gps_callback(data):
    sensor_data['gnss'] = {
        'latitude': spoofed_latitude if spoofing_active else data.latitude,
        'longitude': spoofed_longitude if spoofing_active else data.longitude
    }

def imu_callback(data):
    sensor_data['imu'] = {
        'accel_x': data.accelerometer.x,
        'accel_y': data.accelerometer.y,
        'accel_z': data.accelerometer.z,
        'gyro_x': data.gyroscope.x,
        'gyro_y': data.gyroscope.y,
        'gyro_z': data.gyroscope.z
    }

def rgb_callback(image):
    global frame_id
    if not (sensor_data['gnss'] and sensor_data['imu']): return

    array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    rgb_img = array[:, :, :3][:, :, ::-1]

    velocity = vehicle.get_velocity()
    speed = 3.6 * (velocity.x**2 + velocity.y**2 + velocity.z**2)**0.5

    control = vehicle.get_control()
    features = np.array([
        sensor_data['imu']['accel_x'],
        sensor_data['imu']['accel_y'],
        sensor_data['imu']['accel_z'],
        sensor_data['imu']['gyro_x'],
        sensor_data['imu']['gyro_y'],
        sensor_data['imu']['gyro_z'],
        speed,
        control.steer,
        control.throttle,
        control.brake
    ], dtype=np.float32)

    with torch.no_grad():
        input_tensor = torch.tensor(features).unsqueeze(0).to(DEVICE)
        logits = model(input_tensor)
        probs = torch.softmax(logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        confidence = probs[0, pred].item()

    prediction_label = 'spoofed' if pred == 1 else 'normal'
    print(f"[Frame {frame_id}] Prediction: {prediction_label} ({confidence:.2f})")

    if prediction_label == 'spoofed':
        vehicle.set_autopilot(False)
        brake_control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0)
        vehicle.apply_control(brake_control)
    else:
        vehicle.set_autopilot(True)

    text = f"Prediction: {prediction_label.upper()} ({confidence*100:.1f}%)"
    color = (0, 0, 255) if prediction_label == 'spoofed' else (0, 255, 0)
    cv2.putText(rgb_img, text, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

    if 'rgb' not in video_writers:
        path = 'video_Aml/rgb_video_Aml.avi'
        video_writers['rgb'] = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'XVID'), video_fps, (image.width, image.height))
    video_writers['rgb'].write(rgb_img)

    record = {
        'frame_id': frame_id,
        'image_num': image.frame,
        'latitude': sensor_data['gnss']['latitude'],
        'longitude': sensor_data['gnss']['longitude'],
        'accel_x': features[0],
        'accel_y': features[1],
        'accel_z': features[2],
        'gyro_x': features[3],
        'gyro_y': features[4],
        'gyro_z': features[5],
        'speed': features[6],
        'steering_angle': features[7],
        'throttle': features[8],
        'brake': features[9],
        'label': 'spoofed' if spoofing_active else 'normal',
        'predicted_label': prediction_label,
        'confidence_score': round(confidence, 4)
    }
    data_records.append(record)
    frame_id += 1

# === SENSOR ATTACHMENT ===
gnss = world.spawn_actor(blueprints.find('sensor.other.gnss'), carla.Transform(), attach_to=vehicle)
imu = world.spawn_actor(blueprints.find('sensor.other.imu'), carla.Transform(), attach_to=vehicle)
camera_bp = blueprints.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

gnss.listen(gps_callback)
imu.listen(imu_callback)
camera.listen(rgb_callback)

sensor_list += [gnss, imu, camera]


In [ ]:

# === SPOOFING CONTROL ===
print("\n=== Spoofing Control ===")
print("Type 's' + ENTER → Activate spoofing")
print("Type 'n' + ENTER → Deactivate spoofing")
print("Type 'q' + ENTER → Quit simulation\n")

def spoofing_input_loop():
    global spoofing_active, running
    while running:
        command = input("Command (s=spoof, n=normal, q=quit): ").strip().lower()
        if command == 's':
            spoofing_active = True
            print("🚨 Spoofing Activated.")
        elif command == 'n':
            spoofing_active = False
            print("✅ Spoofing Deactivated.")
        elif command == 'q':
            print("🛑 Quitting simulation.")
            running = False
            break

running = True
input_thread = threading.Thread(target=spoofing_input_loop)
input_thread.start()

try:
    while running and vehicle.is_alive:
        world.tick()
        time.sleep(0.05)
except KeyboardInterrupt:
    print("🔁 Simulation interrupted by user.")
    running = False

input_thread.join()


In [ ]:

# === CLEANUP & SAVE ===
try:
    pd.DataFrame(data_records).to_csv('combined_data_Aml.csv', index=False)
    print("💾 combined_data_Aml.csv saved.")
except Exception as e:
    print(f"Error saving CSV: {e}")

for s in sensor_list:
    try:
        s.stop()
        s.destroy()
    except:
        pass

try:
    vehicle.destroy()
except:
    pass

for writer in video_writers.values():
    try:
        writer.release()
    except:
        pass

print("✅ Data and videos saved. All actors cleaned up.")
